In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Datagens import *
from AttentionUnet import *
from metrics import *

# ml libs
from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_attention_unet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

 conv2d_17 (Conv2D)             (None, 16, 16, 64)   8256        ['conv2d_16[0][0]']              
                                                                                                  
 activation_3 (Activation)      (None, 16, 16, 64)   0           ['conv2d_17[0][0]']              
                                                                                                  
 conv2d_19 (Conv2D)             (None, 16, 16, 64)   4160        ['activation_3[0][0]']           
                                                                                                  
 conv2d_transpose_1 (Conv2DTran  (None, 16, 16, 64)  36928       ['conv2d_19[0][0]']              
 spose)                                                                                           
                                                                                                  
 conv2d_18 (Conv2D)             (None, 16, 16, 64)   16448       ['conv2d_5[0][0]']               
          

                                                                                                  
 conv2d_30 (Conv2D)             (None, 64, 64, 32)   9248        ['dropout_4[0][0]']              
                                                                                                  
 conv2d_31 (Conv2D)             (None, 64, 64, 16)   528         ['conv2d_30[0][0]']              
                                                                                                  
 activation_9 (Activation)      (None, 64, 64, 16)   0           ['conv2d_31[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)             (None, 64, 64, 16)   272         ['activation_9[0][0]']           
                                                                                                  
 conv2d_transpose_3 (Conv2DTran  (None, 64, 64, 16)  2320        ['conv2d_33[0][0]']              
 spose)   

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [5]:
csv_logger = CSVLogger('Logs/training/attU_net_2d.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/attU_net_2d.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [6]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/attU_net_2d.h5")

Epoch 1/100
200/200 [==============================] - 83s 361ms/step - loss: 0.1073 - accuracy: 0.9780 - mean_io_u: 0.7146 - dice_coef: 0.2760 - dice_coef_necrotic: 0.0862 - dice_coef_edema: 0.1378 - dice_coef_enhancing: 0.0575 - val_loss: 0.2313 - val_accuracy: 0.9566 - val_mean_io_u: 0.3756 - val_dice_coef: 0.2668 - val_dice_coef_necrotic: 0.1040 - val_dice_coef_edema: 0.2325 - val_dice_coef_enhancing: 0.1109 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0537 - accuracy: 0.9851 - mean_io_u: 0.7746 - dice_coef: 0.3321 - dice_coef_necrotic: 0.1883 - dice_coef_edema: 0.2814 - dice_coef_enhancing: 0.1710 - val_loss: 0.1071 - val_accuracy: 0.9653 - val_mean_io_u: 0.7782 - val_dice_coef: 0.3233 - val_dice_coef_necrotic: 0.1888 - val_dice_coef_edema: 0.3021 - val_dice_coef_enhancing: 0.2139 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 57s 286ms/step - loss: 0.0417 - accuracy: 0.9847 - mean_io_u: 0.8010 - dice_coef: 0.360

Epoch 17/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0305 - accuracy: 0.9892 - mean_io_u: 0.8077 - dice_coef: 0.4177 - dice_coef_necrotic: 0.2615 - dice_coef_edema: 0.5721 - dice_coef_enhancing: 0.2917 - val_loss: 0.1226 - val_accuracy: 0.9645 - val_mean_io_u: 0.8115 - val_dice_coef: 0.3544 - val_dice_coef_necrotic: 0.1680 - val_dice_coef_edema: 0.3639 - val_dice_coef_enhancing: 0.1764 - lr: 1.0000e-06
Epoch 18/100
200/200 [==============================] - 57s 287ms/step - loss: 0.0304 - accuracy: 0.9892 - mean_io_u: 0.8078 - dice_coef: 0.4178 - dice_coef_necrotic: 0.2614 - dice_coef_edema: 0.5733 - dice_coef_enhancing: 0.2919 - val_loss: 0.1198 - val_accuracy: 0.9662 - val_mean_io_u: 0.8120 - val_dice_coef: 0.3564 - val_dice_coef_necrotic: 0.1680 - val_dice_coef_edema: 0.3669 - val_dice_coef_enhancing: 0.1768 - lr: 1.0000e-06
Epoch 19/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0304 - accuracy: 0.9892 - mean_io_u: 0.8078 - dice_

Epoch 36/100
200/200 [==============================] - 57s 283ms/step - loss: 0.0303 - accuracy: 0.9893 - mean_io_u: 0.8097 - dice_coef: 0.4191 - dice_coef_necrotic: 0.2621 - dice_coef_edema: 0.5756 - dice_coef_enhancing: 0.2946 - val_loss: 0.1239 - val_accuracy: 0.9641 - val_mean_io_u: 0.8142 - val_dice_coef: 0.3549 - val_dice_coef_necrotic: 0.1674 - val_dice_coef_edema: 0.3643 - val_dice_coef_enhancing: 0.1762 - lr: 1.0000e-06
Epoch 37/100
200/200 [==============================] - 57s 286ms/step - loss: 0.0302 - accuracy: 0.9893 - mean_io_u: 0.8098 - dice_coef: 0.4191 - dice_coef_necrotic: 0.2616 - dice_coef_edema: 0.5761 - dice_coef_enhancing: 0.2945 - val_loss: 0.1265 - val_accuracy: 0.9621 - val_mean_io_u: 0.8144 - val_dice_coef: 0.3528 - val_dice_coef_necrotic: 0.1683 - val_dice_coef_edema: 0.3591 - val_dice_coef_enhancing: 0.1773 - lr: 1.0000e-06
Epoch 38/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0303 - accuracy: 0.9893 - mean_io_u: 0.8100 - dice_

Epoch 55/100
200/200 [==============================] - 57s 285ms/step - loss: 0.0301 - accuracy: 0.9893 - mean_io_u: 0.8122 - dice_coef: 0.4204 - dice_coef_necrotic: 0.2622 - dice_coef_edema: 0.5793 - dice_coef_enhancing: 0.2979 - val_loss: 0.1208 - val_accuracy: 0.9662 - val_mean_io_u: 0.8191 - val_dice_coef: 0.3582 - val_dice_coef_necrotic: 0.1664 - val_dice_coef_edema: 0.3698 - val_dice_coef_enhancing: 0.1761 - lr: 1.0000e-06
Epoch 56/100
200/200 [==============================] - 57s 285ms/step - loss: 0.0301 - accuracy: 0.9893 - mean_io_u: 0.8127 - dice_coef: 0.4206 - dice_coef_necrotic: 0.2619 - dice_coef_edema: 0.5805 - dice_coef_enhancing: 0.2967 - val_loss: 0.1262 - val_accuracy: 0.9627 - val_mean_io_u: 0.8190 - val_dice_coef: 0.3542 - val_dice_coef_necrotic: 0.1680 - val_dice_coef_edema: 0.3611 - val_dice_coef_enhancing: 0.1776 - lr: 1.0000e-06
Epoch 57/100
200/200 [==============================] - 57s 286ms/step - loss: 0.0300 - accuracy: 0.9893 - mean_io_u: 0.8125 - dice_

Epoch 74/100
200/200 [==============================] - 57s 284ms/step - loss: 0.0299 - accuracy: 0.9893 - mean_io_u: 0.8145 - dice_coef: 0.4217 - dice_coef_necrotic: 0.2624 - dice_coef_edema: 0.5840 - dice_coef_enhancing: 0.3003 - val_loss: 0.1253 - val_accuracy: 0.9634 - val_mean_io_u: 0.8215 - val_dice_coef: 0.3557 - val_dice_coef_necrotic: 0.1678 - val_dice_coef_edema: 0.3630 - val_dice_coef_enhancing: 0.1781 - lr: 1.0000e-06
Epoch 75/100
200/200 [==============================] - 57s 283ms/step - loss: 0.0298 - accuracy: 0.9893 - mean_io_u: 0.8148 - dice_coef: 0.4218 - dice_coef_necrotic: 0.2623 - dice_coef_edema: 0.5830 - dice_coef_enhancing: 0.3006 - val_loss: 0.1209 - val_accuracy: 0.9664 - val_mean_io_u: 0.8226 - val_dice_coef: 0.3594 - val_dice_coef_necrotic: 0.1660 - val_dice_coef_edema: 0.3710 - val_dice_coef_enhancing: 0.1762 - lr: 1.0000e-06
Epoch 76/100
200/200 [==============================] - 57s 284ms/step - loss: 0.0298 - accuracy: 0.9893 - mean_io_u: 0.8151 - dice_

Epoch 93/100
200/200 [==============================] - 58s 290ms/step - loss: 0.0296 - accuracy: 0.9894 - mean_io_u: 0.8149 - dice_coef: 0.4230 - dice_coef_necrotic: 0.2627 - dice_coef_edema: 0.5871 - dice_coef_enhancing: 0.3037 - val_loss: 0.1265 - val_accuracy: 0.9627 - val_mean_io_u: 0.8223 - val_dice_coef: 0.3553 - val_dice_coef_necrotic: 0.1673 - val_dice_coef_edema: 0.3620 - val_dice_coef_enhancing: 0.1780 - lr: 1.0000e-06
Epoch 94/100
200/200 [==============================] - 57s 287ms/step - loss: 0.0296 - accuracy: 0.9894 - mean_io_u: 0.8149 - dice_coef: 0.4231 - dice_coef_necrotic: 0.2627 - dice_coef_edema: 0.5879 - dice_coef_enhancing: 0.3031 - val_loss: 0.1214 - val_accuracy: 0.9660 - val_mean_io_u: 0.8223 - val_dice_coef: 0.3590 - val_dice_coef_necrotic: 0.1675 - val_dice_coef_edema: 0.3686 - val_dice_coef_enhancing: 0.1784 - lr: 1.0000e-06
Epoch 95/100
200/200 [==============================] - 57s 285ms/step - loss: 0.0295 - accuracy: 0.9894 - mean_io_u: 0.8149 - dice_

In [ ]:
# model = keras.models.load_model('Models/2D/attU_net_2d.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=True)

In [7]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

36/36 [==============================] - 11s 293ms/step - loss: 0.1399 - accuracy: 0.9557 - mean_io_u: 0.8152 - dice_coef: 0.3551 - dice_coef_necrotic: 0.1708 - dice_coef_edema: 0.3561 - dice_coef_enhancing: 0.2182


[0.13991454243659973,
 0.955700695514679,
 0.8151941299438477,
 0.35510438680648804,
 0.1707843542098999,
 0.35608401894569397,
 0.21818523108959198]

In [8]:
valid_csv_logger = CSVLogger('Logs/validation/attU_net_2d.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

60/60 [==============================] - 12s 196ms/step - loss: 0.1232 - accuracy: 0.9649 - mean_io_u: 0.8214 - dice_coef: 0.3576 - dice_coef_necrotic: 0.1679 - dice_coef_edema: 0.3655 - dice_coef_enhancing: 0.1791


[0.12318429350852966,
 0.9648918509483337,
 0.821353554725647,
 0.357617050409317,
 0.16789422929286957,
 0.3655255436897278,
 0.17907029390335083]